In [1]:
from __future__ import print_function
import numpy as np
import matplotlib.pyplot as plt
import scipy.linalg as la
from matplotlib import cm
from numpy.linalg import inv
import random
import math

## WZ term to be calculated 


In [2]:
def eig_(M, sortby = 'real'):
    """ Returns the eigenvalues, eigenvectors (left and right if the matrix is
    non-symmetric) of a matrix M in an ascending order. sortby = 'real', 'imag' or 'abs'. """
    if np.allclose(M, M.T.conj()):
        evals, evecs = la.eig(M)
        sort_p = (evals.real).argsort()
        evals=evals[sort_p]
        evecs=evecs[:, sort_p]
        return evals, evecs
    else:
        evals, evecs_l, evecs_r = la.eig(M, left = True, right = True)
        if sortby == 'real':
           sort_p = (evals.real).argsort()
        elif sortby == 'imag':
           sort_p = (evals.imag).argsort()
        elif sortby == 'abs':
           sort_p = (np.abs(evals)).argsort()
        evals = evals[sort_p]
        evecs_r = evecs_r[:, sort_p]
        evecs_l = evecs_l[:, sort_p]
        return evals, evecs_l, evecs_r


In [4]:
# WZ integral for various bloch matrices depending on lambdas 
sigma_z = np.zeros((4,4), dtype=np.complex128)
sigma_z[0,0] += 1
sigma_z[1,1] += 1
sigma_z[2,2] += -1
sigma_z[3,3] += -1

# Bloch Hamiltonian
n_k=23 # number of steps 
k_x = np.linspace(0,2*np.pi,n_k, endpoint=False)
k_y = np.linspace(0,2*np.pi,n_k, endpoint=False)
k_z = np.linspace(0,2*np.pi,n_k, endpoint=False)
gamma=0.5
lamb=1
H = np.zeros((4,4), dtype=np.complex128)
n_t = n_k # number of steps
t = np.linspace(0,1,n_t)

sigma_x = np.array([[0,1],[1,0]])
sigma_y = np.array([[0,-1j],[1j,0]])
sigma_z = np.array([[1,0],[0,-1]])
I = np.identity(2)


def Bloch_pauli(kx, ky, kz, gamma, lamb):
    H_1 = 0#1j*lamb*np.sin(ky)*np.kron(sigma_x,I)+1j*(gamma+lamb*np.cos(ky))*np.kron(sigma_y,I)
    H_2 = 1j*lamb*np.sin(kx)*np.kron(sigma_z,I)+(gamma+lamb*np.cos(kx))*np.kron(I,sigma_z)#1j*lamb*np.sin(kx)*np.kron(sigma_z,I)+
    H_3 = lamb*np.sin(kz)*np.kron(I,sigma_y)+(gamma+lamb*np.cos(kz))*np.kron(I,sigma_x)
    H_p = H_1+H_2+H_3
    #Hermitized-------------
    Hermi=(H_p+H_p.conj().T)/2
    #-----------------------
    return H_p


In [5]:
#from here Fourier transform 
a1=np.array([1.,0.,0.])
a2=np.array([0.,1.,0.])
a3=np.array([0.,0.,1.])

#-------------------------------------------------------------
hopping=np.zeros((7,4,4),dtype=np.complex128)


for i in range(len(k_x)):
    for r in range(len(k_z)):
        ky=np.pi
        k = np.array([k_x[i],ky,k_z[r]])
        hopping[0] += (Bloch_pauli(k_x[i],ky,k_z[r], gamma,lamb))
        hopping[1] += (np.exp(1j*np.dot(k,a1))*Bloch_pauli(k_x[i],ky,k_z[r],gamma,lamb))
        hopping[2] += (np.exp(-1j*np.dot(k,a1))*Bloch_pauli(k_x[i],ky,k_z[r],gamma,lamb))
        hopping[5] += (np.exp(1j*np.dot(k,a3))*Bloch_pauli(k_x[i],ky,k_z[r],gamma,lamb))
        hopping[6] += (np.exp(-1j*np.dot(k,a3))*Bloch_pauli(k_x[i],ky,k_z[r],gamma,lamb))

hopping[0]=hopping[0]/(n_k*n_k)
hopping[1]=hopping[1]/(n_k*n_k)
hopping[2]=hopping[2]/(n_k*n_k)
hopping[5]=hopping[5]/(n_k*n_k)
hopping[6]=hopping[6]/(n_k*n_k)

print('0',np.round(hopping[0],2))
print('a1',np.round(hopping[1],2))
print('-a1',np.round(hopping[2],2))
print('a3',np.round(hopping[5],2))
print('-a3',np.round(hopping[6],2))


0 [[ 0.5+0.j  0.5+0.j  0. +0.j  0. +0.j]
 [ 0.5-0.j -0.5+0.j  0. +0.j  0. +0.j]
 [ 0. +0.j  0. +0.j  0.5-0.j  0.5+0.j]
 [ 0. +0.j  0. +0.j  0.5-0.j -0.5-0.j]]
a1 [[-0.+0.j -0.-0.j  0.+0.j  0.+0.j]
 [-0.-0.j -1.-0.j  0.+0.j  0.+0.j]
 [ 0.+0.j  0.+0.j  1.+0.j -0.-0.j]
 [ 0.+0.j  0.+0.j -0.-0.j  0.-0.j]]
-a1 [[ 1.-0.j -0.+0.j  0.+0.j  0.+0.j]
 [-0.+0.j  0.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j  0.+0.j -0.-0.j -0.+0.j]
 [ 0.+0.j  0.+0.j -0.+0.j -1.+0.j]]
a3 [[-0.-0.j  1.-0.j  0.+0.j  0.+0.j]
 [-0.+0.j  0.-0.j  0.+0.j  0.+0.j]
 [ 0.+0.j  0.+0.j -0.+0.j  1.-0.j]
 [ 0.+0.j  0.+0.j -0.+0.j  0.+0.j]]
-a3 [[-0.-0.j -0.-0.j  0.+0.j  0.+0.j]
 [ 1.+0.j  0.-0.j  0.+0.j  0.+0.j]
 [ 0.+0.j  0.+0.j -0.+0.j -0.-0.j]
 [ 0.+0.j  0.+0.j  1.+0.j  0.+0.j]]


In [6]:
%matplotlib qt5
#L=2  # number of unit cells
L = 2
xcell = L
zcell = L
X=0 # boundary terms
Z=0

def FT(L):

    L1 = L # number of the unit cell and same for all directions 
    L2 = L
    n=4*L*L
    A = 4
    H=np.zeros((n,n),dtype=np.complex128)                #building real space hamiltonian using FT blocks 

    for i in range(n//4):
        j=A*i
        H[j:j+A,j:j+A]+=hopping[0]
    #print(np.round(H[0:8,0:8],2))
    
    #---------------------------------------------------------------------
  
    m=0  
    for i in range(L2):
        for j in range(L1-1):
            H[m+A*j:m+A*j+A,m+A*j+A:m+A*j+2*A]+=hopping[1]   # in x direction
            H[m+A*j+A:m+A*j+2*A,m+A*j:m+A*j+A]+=hopping[2]
        m+=L1*A
    g=0
    s=L1*A
    for j in range(L2-1):  # 
        for i in range(L1):
            H[g+A*i:g+A*i+A,g+s+A*i:g+s+A*i+A]+=hopping[6]   # in y direction 
            H[g+s+A*i:g+s+A*i+A,g+A*i:g+A*i+A]+=hopping[5]
        g+=L1*A   # g is: every time the indices start from new row 



    #---------------------------------------------------------------------
    #boundary terms in x, y, z directions 

    v1=0
    for i in range(L2): # the length of the y direction is to add the x boundary hoppings 
        H[v1:v1+A,v1+A*(L1-1):v1+A*L1]+=X*hopping[2]   # in x direction
        H[v1+A*(L1-1):v1+A*L1,v1:v1+A]+=X*hopping[1]
        v1+=L1*A


    v2=0
    for i in range(L1):# the length of the x direction is to add the y boundary hoppings
        H[v2:v2+A,v2+A*L1*(L2-1):v2+A*L1*(L2-1)+A]+=Z*hopping[5]   # in y direction
        H[v2+A*L1*(L2-1):v2+A*L1*(L2-1)+A,v2:v2+A]+=Z*hopping[6]
        v2+=A


    return H



print(np.round(FT(4)[8:16,0:8],2))


[[ 0.+0.j  0.+0.j  0.+0.j  0.+0.j  1.-0.j -0.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j  0.+0.j  0.+0.j  0.+0.j -0.+0.j  0.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j -0.-0.j -0.+0.j]
 [ 0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j -0.+0.j -1.+0.j]
 [ 0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j]]


In [7]:
L=20
e = eig_(FT(L))  #evals, evecs_l, evecs_r


In [8]:
#calculate IPR for the right vector 
%matplotlib qt5
A=4
n=A*L*L
sumRight = np.zeros(n)
l=0
while l<n:
    index = l
    vecpropl = e[1][:,index] #label r for right vector l for left
    vecpropr = e[1][:,index] #label r for right vector l for left
    psiSqr=vecpropr*np.conj(vecpropr) #probability for every lattice site
    sumRight[index] = sum(np.square(psiSqr))
    l += 1
    

/Users/anar/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:12: ComplexWarning: Casting complex values to real discards the imaginary part
  if sys.path[0] == '':


In [9]:
%matplotlib qt5
plt.style.use('ggplot')
length=np.where(sumRight>0.2)
print(len(length[0]))

#print(sumRight[440])
#print(sumRight[100])
print(length)
num_bins = 60
plt.hist(sumRight, num_bins,range=[0, 1], facecolor='magenta', alpha=0.5)
plt.title(r'Hermitized; FT along $k_x$ and $k_z$; $k_y=\pi$; unit cell: {}'.format(L))
plt.xlabel('Localization')
plt.ylabel('# of incidents')
#plt.ylim([0,700])
plt.show()


80
(array([740, 741, 742, 743, 748, 749, 750, 751, 756, 757, 758, 759, 768,
       769, 770, 771, 776, 777, 778, 779, 780, 781, 782, 783, 784, 785,
       786, 787, 788, 789, 790, 791, 792, 793, 794, 795, 796, 797, 798,
       799, 800, 801, 802, 803, 804, 805, 806, 807, 808, 809, 810, 811,
       812, 813, 814, 815, 816, 817, 818, 819, 820, 821, 822, 823, 828,
       829, 830, 831, 840, 841, 842, 843, 848, 849, 850, 851, 856, 857,
       858, 859]),)


In [10]:
%matplotlib qt5
yatoms=4*L
xatoms=L
n=yatoms*xatoms
s=0
probDensity = np.zeros((n,n))
while s<n:
    index = s
    vecpropr = e[1][:,index] #label r for right vector
    psiSqr=vecpropr*np.conj(vecpropr) #probability for every lattice site
    for j in range(len(probDensity[0])):
        probDensity[index][j]+=psiSqr[j]
    s += 1

#print(len(probDensity))

/Users/anar/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:12: ComplexWarning: Casting complex values to real discards the imaginary part
  if sys.path[0] == '':


In [11]:
%matplotlib qt5
state=801

# Implementation of matplotlib function 
dx, dy = 0.015, 0.015
y, x = np.mgrid[slice(0, 20 + dy, dy), 
                slice(0, 20 + dx, dx)] 


plt.title(r'FT along $k_x$ and $k_z$;keeping nH-$k_x$ terms; unit cell: {}'.format(L))


z=probDensity[state].reshape((xatoms,yatoms))

z_min, z_max = z.min(), z.max() 

c = plt.imshow(z, cmap ='YlOrBr', vmin = z_min, vmax = z_max, 
                 extent =[x.min(), x.max(), y.min(), y.max()], 
                    interpolation ='nearest', origin ='lower') 
#plt.clim(0.0,0.30)
plt.colorbar(c)

In [106]:
%matplotlib qt5
states = np.array([740, 741, 742, 743, 748, 749, 750, 751, 756, 757, 758, 759, 768,
       769, 770, 771, 776, 777, 778, 779, 780, 781, 782, 783, 784, 785,
       786, 787, 788, 789, 790, 791, 792, 793, 794, 795, 796, 797, 798,
       799, 800, 801, 802, 803, 804, 805, 806, 807, 808, 809, 810, 811,
       812, 813, 814, 815, 816, 817, 818, 819, 820, 821, 822, 823, 828,
       829, 830, 831, 840, 841, 842, 843, 848, 849, 850, 851, 856, 857,
       858, 859])

# Implementation of matplotlib function 
dx, dy = 0.015, 0.015
y, x = np.mgrid[slice(0, 20 + dy, dy), 
                slice(0, 20 + dx, dx)] 


plt.title(r'added;FT along $k_x$ and $k_z$;keeping nH-$k_x$; unit cell: {}'.format(L))
z=np.zeros(len(probDensity[state])).reshape((xatoms,yatoms))

for i in states:
    z+=probDensity[i].reshape((xatoms,yatoms))
z=z/len(states)
z_min, z_max = z.min(), z.max() 

c = plt.imshow(z, cmap ='YlOrBr', vmin = z_min, vmax = z_max, 
                 extent =[x.min(), x.max(), y.min(), y.max()], 
                    interpolation ='nearest', origin ='lower') 
#plt.clim(0.0,0.30)
plt.colorbar(c)

In [100]:
%matplotlib qt5
plt.style.use('ggplot')

plt.scatter(e[0].real, e[0].imag, s=10, c=sumRight, cmap='viridis',vmin=0)#,vmax=0.2)

plt.title('Energy spectra')

plt.colorbar()
plt.xlim([-3, 3])
plt.ylim([-3, 3])
plt.xlabel('Energy, real')
plt.ylabel('Energy, imag')

plt.show()



In [99]:
%matplotlib qt5
plt.style.use('ggplot')

n=4*L*L 
x=np.linspace(0,n,n)
plt.scatter(x, e[0].imag, s=10, c=sumRight, cmap='viridis',vmin=0)#,vmax=0.2)

plt.title('Spectra')
plt.colorbar()
#plt.xlim([-2, 2])
#plt.ylim([-2, 2])
plt.xlabel('unit cell')
plt.ylabel('Energy, real')

plt.show()
